<a href="https://colab.research.google.com/github/scgupta/yearn2learn/blob/master/speech/asr/python_speech_recognition_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speech Recognition in Python

There are several Automated Speech Recognition (ASR) alternatives, and most of them have bindings for Python. There are two kinds of solutions:

- **Online:** These run on the cloud, and are accessed either through REST endpoints or Python library. Examples are services from Google, Amazon, Microsoft.
- **Offline:** These run locally on the machine (i.e. no network connection required). These comes with flexibility or overhead (depending upon perspective) of needing to have either download pre-trained models, or you can train on your data as perneed. Examples are [CMU Sphinx](https://cmusphinx.github.io/), [Mozilla DeepSpeech](https://hacks.mozilla.org/2019/12/deepspeech-0-6-mozillas-speech-to-text-engine/), [Facebook wav2letter@anywhere](https://ai.facebook.com/blog/online-speech-recognition-with-wav2letteranywhere/).

Speech Recognition APIs are of two types:
- **Batch:** The full audio file is passed as parameter, and speech-to-text transcribing is done in one shot.
- **Streaming:** The chunks of audio buffer are repeatedly passed on, and intermediate results are accessible.

All packages support batch mode, and some support streaming mode too.

One common use case is to collect audio from microphone and passes on the buffer to the speech recognition API. Invariably, in such transcribers, microphone is accessed though [PyAudio](https://people.csail.mit.edu/hubert/pyaudio/), which is implemented over [PortAudio](http://www.portaudio.com/).

From Colab menu, select: **Runtime** > **Change runtime type**, and verify that it is set to Python3, and select GPU if you want to try out GPU version.

## Download audio files and create test cases

Following files will be used as test cases for all ASR packages.

In [0]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/audio-0.6.0.tar.gz
!tar -xvzf audio-0.6.0.tar.gz
!ls -l ./audio/

In [0]:
TESTCASES = [
  {
    'filename': 'audio/2830-3980-0043.wav',
    'text': 'experience proves this',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/4507-16021-0012.wav',
    'text': 'why should one halt on the way',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/8455-210777-0068.wav',
    'text': 'your power is sufficient i said',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  }
]

## Utility Functions

In [0]:
from typing import Tuple
import wave

def read_wav_file(filename) -> Tuple[bytes, int]:
    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

def simulate_stream(buffer: bytes, batch_size: int = 4096):
    buffer_len = len(buffer)
    offset = 0
    while offset < buffer_len:
        end_offset = offset + batch_size
        buf = buffer[offset:end_offset]
        yield buf
        offset = end_offset


---

# Google

Google has only online speech-to-text ([documentation](https://cloud.google.com/speech-to-text/docs)) and supports both batch and stream modes.

## Setup

1. **Install google cloud speech package**

In [0]:
!pip3 install google-cloud-speech

2. **Upload Google Cloud Cred file**

Have Google Cloud creds stored in a file named `gc-creds.json`, and upload it by running following code cell. See https://developers.google.com/accounts/docs/application-default-credentials for more details.

This may reqire enabling **third-party cookies**. Check out https://colab.research.google.com/notebooks/io.ipynb for other alternatives.

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!pwd
!ls -l ./gc-creds.json

In [0]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gc-creds.json'

!ls -l $GOOGLE_APPLICATION_CREDENTIALS

## Batch API

In [0]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def google_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    client = speech_v1.SpeechClient()

    config = {
        'language_code': lang,
        'sample_rate_hertz': rate,
        'encoding': enums.RecognitionConfig.AudioEncoding[encoding]
    }

    audio = {
        'content': buffer
    }

    response = client.recognize(config, audio)
    # For bigger audio file, the previous line can be replaced with following:
    # operation = client.long_running_recognize(config, audio)
    # response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        return alternative.transcript

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-batch-stt: "{}"'.format(
        google_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))

## Streaming API

In [0]:
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

def response_stream_processor(responses):
    print('interim results: ')

    transcript = ''
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        result = response.results[0]
        if not result.alternatives:
            continue

        transcript = result.alternatives[0].transcript
        print('{0}final: {1}'.format(
            '' if result.is_final else 'not ',
            transcript
        ))

    return transcript

def google_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)

    client = speech.SpeechClient()

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding[encoding],
        sample_rate_hertz=rate,
        language_code=lang
    )

    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True
    )

    audio_generator = simulate_stream(buffer)  # buffer chunk generator
    requests = (types.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator)
    responses = client.streaming_recognize(streaming_config, requests)
    # Now, put the transcription responses to use.
    return response_stream_processor(responses)

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-streaming-stt: "{}"'.format(
        google_streaming_stt(t['filename'], t['lang'], t['encoding'])
    ))


---

# Amazon


---

# Microsoft Azure


---

# CMU Sphinx


---

# Mozilla DeepSpeech

Mozilla released [DeepSpeech 0.6](https://github.com/mozilla/DeepSpeech/releases/tag/v0.6.0) with [APIs in C, Java, .NET, Python, and JavaScript](https://deepspeech.readthedocs.io/en/v0.6.0/Python-API.html).

## Setup

1. **Install DeepSpeech**

You can install DeepSpeech with pip (make it deepspeech-gpu==0.6.0 if you are using GPU in colab runtime).

In [0]:
!pip install deepspeech==0.6.0

2. **Download and unzip models**

In [0]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-models.tar.gz
!tar -xvzf deepspeech-0.6.0-models.tar.gz
!ls -l ./deepspeech-0.6.0-models/

3. **Test that it all works**

Examine the output of the last three commands, and you will see results *“experience proof less”*, *“why should one halt on the way”*, and *“your power is sufficient i said”* respectively. You are all set.

In [0]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.6.0-models/lm.binary --trie ./deepspeech-0.6.0-models/trie --audio ./audio/2830-3980-0043.wav

In [0]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.6.0-models/lm.binary --trie ./deepspeech-0.6.0-models/trie --audio ./audio/4507-16021-0012.wav

In [0]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.6.0-models/lm.binary --trie ./deepspeech-0.6.0-models/trie --audio ./audio/8455-210777-0068.wav

## API: Create model object

In [0]:
import deepspeech

model_file_path = 'deepspeech-0.6.0-models/output_graph.pbmm'
beam_width = 500
model = deepspeech.Model(model_file_path, beam_width)

# Add language model for better accuracy
lm_file_path = 'deepspeech-0.6.0-models/lm.binary'
trie_file_path = 'deepspeech-0.6.0-models/trie'
lm_alpha = 0.75
lm_beta = 1.85
model.enableDecoderWithLM(lm_file_path, trie_file_path, lm_alpha, lm_beta)

## Batch API

In [0]:
import numpy as np

def deepspeech_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    data16 = np.frombuffer(buffer, dtype=np.int16)
    return model.stt(data16)

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('deepspeech-batch-stt: "{}"'.format(
        deepspeech_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))

## Streaming API

In [0]:
def deepspeech_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    audio_generator = simulate_stream(buffer)

    # Create stream
    context = model.createStream()

    text = ''
    for chunk in audio_generator:
        data16 = np.frombuffer(chunk, dtype=np.int16)
        # feed stream of chunks
        model.feedAudioContent(context, data16)
        interim_text = model.intermediateDecode(context)
        if interim_text != text:
            text = interim_text
            print('inetrim result: {}'.format(text))

    # get final resut and close stream
    text = model.finishStream(context)
    return text

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('deepspeech-streaming-stt: "{}"'.format(
        deepspeech_streaming_stt(t['filename'], t['lang'], t['encoding'])
    ))


---

# Facebook wav2letter@anywhere


---

# SpeechRecognition Package

The [SpeechRecognition](https://pypi.org/project/SpeechRecognition/) package provide a nice abstraction over several packages. In this notebook we explore using CMU Sphinx (an offline model, i.e. running locally), and Google (an offline model, i.e. over the network/cloud), but through SpeechRecognition package APIs.

## Setup

We need to install SpeechRecognition and pocketsphinx python packages, and download some files to test these APIs.

1. **Install SpeechRecognition py package**

In [0]:
!pip3 install SpeechRecognition

2. **Install Pocketsphinx**

[Pocketsphinx](https://pypi.org/project/pocketsphinx/) is python bindings for [CMU Sphinx](https://cmusphinx.github.io/), and is one of the recognizer supported by SpeechRecognition.

On MacOS: (using homebew):

In [0]:
!brew install swig
!swig -version

On Linux:

In [0]:
!apt-get install -y swig libpulse-dev
!swig -version

Now pip install poocketsphinx

In [0]:
!pip3 install pocketsphinx
!pip3 list | grep pocketsphinx

## SpeechRecognition API

SpeechRecognition has only batch API. First step to create an audio record, eithher from a file or from mic, and the second step is to call `recognize_<speech engine name>()` function. It currently has APIs for [CMU Sphinx, Google, Microsoft, IBM, Houndify, and Wit](https://github.com/Uberi/speech_recognition).

In [0]:
import speech_recognition as sr
from enum import Enum, unique


@unique
class ASREngine(Enum):
    sphinx = 0
    google = 1


def speech_to_text(filename: str, engine: ASREngine, language: str, show_all: bool = False) -> str:
    r = sr.Recognizer()

    with sr.AudioFile(filename) as source:
        audio = r.record(source)

    asr_functions = {
        ASREngine.sphinx: r.recognize_sphinx,
        ASREngine.google: r.recognize_google,
    }

    response = asr_functions[engine](audio, language=language, show_all=show_all)
    return response


# Run tests
for t in TESTCASES:
    filename = t['filename']
    text = t['text']
    lang = t['lang']

    print('\naudio file="{0}"    expected text="{1}"'.format(filename, text))
    for asr_engine in ASREngine:
        try:
            response = speech_to_text(filename, asr_engine, language=lang)
            print('{0}: "{1}"'.format(asr_engine.name, response))
        except sr.UnknownValueError:
            print('{0} could not understand audio'.format(asr_engine.name))
        except sr.RequestError as e:
            print('{0} error: {0}'.format(asr_engine.name, e))

Notice the difference in results from sniphix and google.

## API for other providers

For other speech recognition providers, you will need to create API credentials, which you have to pass to `recognize_<speech engine name>()`, you can checkout [this example](https://github.com/Uberi/speech_recognition/blob/master/examples/audio_transcribe.py).

It also has a nice abstraction for Microphone, implemented over PyAudio/PortAudio. Here is an example to capture input from mic in [batch](https://github.com/Uberi/speech_recognition/blob/master/examples/microphone_recognition.py) and continously in [background](https://github.com/Uberi/speech_recognition/blob/master/examples/background_listening.py).


---

# Summary

You are not master of APIs of various speech recognition packages.
